# 206. Reverse Linked List

[link](https://leetcode.com/problems/reverse-linked-list/description/)

## Understand the task

We are given a singly-linked list, which means that each node only knows what the next node in the chain is:


In [1]:
class ListNode(object):
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

Each node has a value, but the values are not guaranteed to be unique. So, we can only use pointers to tell the nodes apart.

The task states that the algorithm can be implemented iteratively or recursively. We can try to implement both and see which implementation has better performance.

Lastly, we are asked to return the head of the new list, so we will need a special pointer for that.

## Brute-force solution

To reverse a list, we need to visit every node in the list, create a copy of it and append it to the new list, which we will return at the end.

The complicated part is that we have a singly-linked list, so we can't go back to previous nodes, nor can we insert nodes *before* the node we are currently at. This means that we have to get to the end of the original list first, before we start appending nodes to the new list.

Using recursion seems like a natural solution for this problem: we can chain function calls in a way where the last node is returned first, then the second-to-last node is appended to it, then the third-to last, an so on:

In [2]:
# newList = append(node N, append(node N - 1, ..., append(node 1)))

The base cases for the recursive algorithm are:

* An empty list
* A node with no next node connected to it

The recursive case is when the current node has a `next`.

Now we can start implementing the algorithm:

In [3]:
class Solution0(object):
    def reverseList(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        # Base cases
        if head == None or head.next == None:
            return head
        
        # Recursive case
        else:
            rest = self.reverseList(head.next)
            rest.next = head
            head.next = None
            return rest

When we run this, it works for test cases with 0 and 2 nodes, but for a list with 5 nodes, we only get the first and the last nodes in reversed order; the rest of the nodes are lost:

> Input
>
> head = [1,2,3,4,5]
>
> Output
>
> [5,1]
>
> Expected
>
> [5,4,3,2,1]

This means that we lost pointers to the remaining nodes - most likely when we attached `head` to `rest`.

To fix this, we can use two separate pointers: one to the head of the reversed list, which we will return at the end, the other - to the end, where we want to attach `head`. To keep the reference to the current pointer, we can either create a wrapper for the recursive function, or add another argument to the function, or create an instance variable for the pointer. We will start with an instance variable and try other approaches later, if necessary.

In [ ]:
class Solution1(object):
    def __init__(self):
        self.current = None

    def reverseList(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        # Base cases
        if head == None or head.next == None:
            self.current = head
            return head
        
        # Recursive case
        else:
            rest = self.reverseList(head.next)
            head.next = None
            self.current.next = head
            self.current = head
            return rest

We got a working solution. Now we can look at the performance:

> **Runtime**
>
> 16ms
>
> Beats 74.49% of users with Python
>
> **Memory**
>
> 16.81MB
>
> Beats 10.69% of users with Python
>

## Optimizing for space

Our algorithm takes up a lot of memory space. This can be caused by the recursive calls that take up stack memory space; it may also be caused by the instance variable.

We will try to refactor the instance variable first by using a wrapper function. The caveat in this case is that the recursive function will have to pass both pointers (to the head of the reversed list and to its current node) through all the function calls, which may be counterproductive to reducing memory use.

In [ ]:
class Solution2(object):
    def reverseList(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        (result, current) = self.reverseRest(head)
        return result

    def reverseRest(self, head, current = None):
        # Base cases
        if head == None or head.next == None:
            current = head
            return (head, current)
        
        # Recursive case
        else:
            (rest, newCurrent) = self.reverseRest(head.next, current)
            head.next = None
            newCurrent.next = head
            newCurrent = head
            return (rest, newCurrent)


When we run this code, it confirms that the memory use is actually worse with this approach:

> **Runtime**
>
> 15ms
>
> Beats 79.56% of users with Python
>
> **Memory**
>
> 18.20MB
>
> Beats 6.44% of users with Python

So we can go back to using the instance variable and work on optimizing the stack memory use instead.

To do that, we will try converting our recursive solution into an iterative one, as is suggested in the task.

In the iterative solution, we will be stepping through the list without being able to get return values from consequent iterations. To keep the algorithm space-efficient, we don't want to go through the list more than once. This means that we will have to copy the current head node at every step and insert it at the *start* of the new list, like so:

In [ ]:
# reversedList = append(node N, node N - 1)

In [ ]:
class Solution3(object):
    def reverseList(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        # Trivial cases
        if head == None or head.next == None:
            return head
        
        # Iterative case
        else:
            reversed = None

            while head is not None:
                if reversed is None:
                    reversed = ListNode(head.val)  # 1
                else:
                    newReversed = ListNode(head.val, reversed)   # 2 - 1
                    reversed = newReversed # 2 - 1

                head = head.next

            return reversed

This solution has an even better runtime, but memory use is still pretty high:

>**Runtime**
>
>8ms
>
>Beats 98.39% of users with Python
>
>**Memory**
>
>15.20MB
>
>Beats 16.80% of users with Python
>

One way in which our algorithm uses space is creating new list nodes with the same value as `head` before inserting them at the start of the list. We can try utilizing pointers to existing nodes instead, which is less intuitive, but may help with space efficiency.

In [ ]:
class Solution4(object):
    def reverseList(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        # Trivial cases
        if head == None or head.next == None:
            return head
        
        # Iterative case
        else:
            reversed = None

            while head is not None:
                nextHead = head.next

                if reversed is None:
                    head.next = None
                else:
                    head.next = reversed

                reversed = head
                head = nextHead

            return reversed

This improved memory use significantly, so we can stop at this solution.

> **Runtime**
>
> 14ms
>
> Beats 85.78% of users with Python
>
> **Memory**
>
> 13.64MB
>
> Beats 48.80% of users with Python
>